In [1]:
#some libraries
!pip install -qU \
  sentence-transformers==2.2.2 \
  accelerate==0.21.0 \
  xformers==0.0.20 \
  bitsandbytes==0.41.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 11.2 MB/s eta 0:00:00
  

In [2]:
#specific version to avoid error with vector store
!pip install chromadb==0.5.3
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 63.2 MB/s eta 0:00:0

In [3]:
#update transformers
#!pip install -U transformers

In [5]:
#loading the mistral-7b-instruct model and the tokenizer
from torch import cuda, bfloat16
import transformers

model_id = 'mistralai/Mistral-7B-Instruct-v0.3'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

# begin initializing HF items, need auth token for these
hf_auth = 'hf_token'
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)
model.eval()
print(f"Model loaded on {device}")
#tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Model loaded on cuda:0


tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [6]:
#mounting drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
#loading test data for model evaluation
import pandas as pd
test_data_path='/content/drive/MyDrive/test_data_split.csv'

In [8]:
#processing the test data to add the queries
#defining the queries for the properties-we need to choose a query based on the input
query_temperature="working temperature of the quantum cascade laser"
query_power= "optical power of the quantum cascade laser"
query_frequency="lasing frequency of the quantum cascade laser"
query_hs="heterostructure of the quantum cascade laser"
query_design_type="design type of the quantum cascade laser"

In [9]:
#checking for the query keywords
def check_text(row, column_name):
    text = row[column_name].lower()
    if "working temperature" in text:
        return query_temperature
    elif "optical power" in text:
        return query_power
    elif "lasing frequency" in text:
        return query_frequency
    elif "query_hs" in text:
        return query_hs
    else:
        return query_design_type

In [10]:
#processing the data
def process_csv(input_filepath, output_filepath, column_name):
    # Load the CSV file
    df = pd.read_csv(input_filepath)

    # Check if the specified column exists
    if column_name in df.columns:
        # Apply the check_text function to the specified column and create a new column with the results
        df['queries'] = df.apply(check_text, axis=1, args=(column_name,))
    else:
        print(f"Column '{column_name}' does not exist in the CSV file.")
        return

    # Save the modified DataFrame to a new CSV file
    df.to_csv(output_filepath, index=False)
    print(f"Processed file saved as '{output_filepath}'.")

In [11]:
#processing the test data
input_csv = test_data_path
output_csv = '/content/drive/MyDrive/processed_test_data.csv'
column_to_check = 'Instruction'
process_csv(input_csv, output_csv, column_to_check)

Processed file saved as '/content/drive/MyDrive/processed_test_data.csv'.


In [12]:
#loading the processed test data
test_data_df=pd.read_csv('/content/drive/MyDrive/processed_test_data.csv')

In [13]:
#putting the data in some order and renaming the columns
test_data_df=test_data_df[['queries', 'Instruction', 'input_text']]
test_data_df.rename(columns={'Instruction': 'instructions', 'input_text': 'texts'}, inplace=True)

<ipython-input-13-926f16bad54d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data_df.rename(columns={'Instruction': 'instructions', 'input_text': 'texts'}, inplace=True)


In [14]:
#generating a lsit of prompts
prompts=[]
for a,b in zip(test_data_df['instructions'],test_data_df['texts']):
    prompt = f'''
[INST]
Problem Definition: Extraction of quantum cascade laser properties from text entails extracting properties from a given text description. This should be done without providing any other additional information or explanations.

Instruction: {a}

Input Text: {b}

[/INST]
'''
    prompts.append(prompt)

In [15]:
#verifying the list of prompts generated for each row in the dataset
#display the prompt
#prompts

In [16]:
#verify the length of the prompt list
print(len(prompts))

130


In [17]:
#we now pass the list of prompts to the model to extract responses in an iterative manner.
#getting the output with transformers pipeline
#import pipeline
from transformers import pipeline
prompts=prompts
responses=[] #list to store the responses from the model.
for prompt in prompts:
  pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=800)
  result = pipe(prompt)
  responses.append(result[0]['generated_text'][len(prompt):])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_to

In [18]:
#verify the responses
#print responses
responses

['The lasing frequency of the Quantum Cascade Semiconductor laser device is 3.3 THz.',
 'The optical power of the Quantum Cascade Semiconductor laser device is approximately 10 mW.',
 'The lasing frequency of the Quantum Cascade Semiconductor laser device is 2.07 THz.',
 'The working temperature of the Quantum Cascade Semiconductor laser device is 190 K in pulsed mode and 125 K in continuous wave mode.',
 'The working temperature of the Quantum Cascade Semiconductor laser device in continuous wave (cw) mode is 91 K.',
 'The heterostructure of the Quantum Cascade Semiconductor laser device, as per the given text, is GaAs/Al0.23Ga0.77As.',
 'The heterostructure of the Quantum Cascade Semiconductor laser device, as described in the input text, is In0.6Ga0.4As/InAs.',
 'The lasing frequency of the Quantum Cascade Semiconductor laser device is approximately 3.0 THz.',
 'The working temperature of the Quantum Cascade Semiconductor laser device is 185 K.',
 'The working temperature of the Qua

In [19]:
#writing the model responses to a csv file
import csv

# Name of the csv file to save the list
file_name = "mistral-7b-instruct-responses.csv"

# Writing the list to a csv file with one column
with open(file_name, "w", newline="") as file:
    writer = csv.writer(file)
    for item in responses:
        writer.writerow([item])

print("List saved to", file_name)

List saved to mistral-7b-instruct-responses.csv
